<a href="https://colab.research.google.com/github/Yannethtj/PruebasSaberProInteligenciaArtificial/blob/main/03-preprocesado_%26_RegresionLogistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!kaggle
!chmod 600 ./kaggle.json

usage: kaggle [-h] [-v] [-W] {competitions,c,datasets,d,kernels,k,models,m,files,f,config} ...
kaggle: error: the following arguments are required: command


In [ ]:
!kaggle competitions download -c udea-ai4eng-20242

 70% 14.0M/20.1M [00:00<00:00, 141MB/s]
100% 20.1M/20.1M [00:00<00:00, 167MB/s]


In [ ]:
!unzip udea-ai4eng-20242.zip


Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.classification import setup, compare_models, finalize_model, predict_model
import pandas as pd

# Cargar datasets

# Configurar PyCaret
setup(
    data=train,
    target='RENDIMIENTO_GLOBAL',  # Cambiar 'target' por la columna objetivo
    train_size=0.7,
    session_id=123,
    fold=3,
    verbose=False
)

# Comparar modelos
mejores_modelos = compare_models(sort='Accuracy', n_select=3, cross_validation=False)

# Finalizar y predecir con el mejor modelo
mejor_modelo = finalize_model(mejores_modelos[0])
predicciones = predict_model(mejor_modelo, data=test)

# Ver predicciones
print(predicciones)


Usamos Pycaret para ver cuales serian los mejores modelos para nuestro dataset.
<br/>
y despues de aprox 20 minutos nos presentó que los mejores modelos son xgboost, lightgbm los cuales usaremos para nuestros proximos modelos.

In [ ]:
from sklearn.preprocessing import LabelEncoder

Inicialmente vamos a tratar las columnas que quedamos por organizar en mi preprocesado, primero vamos a usar una función para normalizar el texto y que ya no aparezcan caracteres especiales.
Despues vamos agrupar los valores siguiendo su frecuencia relativa

In [ ]:
def limpiar_y_preprocesar(data):
  # Label encoding
  label_encoder = LabelEncoder()
  new_labels = pd.DataFrame(label_encoder.fit_transform(data['RENDIMIENTO_GLOBAL']), columns=['RENDIMIENTO_GLOBAL'])
  data = pd.concat([data.drop('RENDIMIENTO_GLOBAL', axis=1), new_labels], axis=1)
  data = data.drop('ID', axis=1)
  data['PERIODO'] = data['PERIODO'].astype(str).apply(lambda text: text[:4])
  def normalize_text(text: str):
    import unicodedata
    import re
    if not pd.isna(text):
        # Normaliza el texto eliminando acentos
        normalized = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
        # Elimina caracteres no deseados, incluidos "¿" y "?"
        return re.sub(r'[¿?]', '', normalized)
    return text

  def group_values_by_frequency(data: pd.DataFrame, col: str, thresholds: list[float] = [0.05, 0.2]):
    """
    Agrupa los valores de una columna en función de su frecuencia relativa.
    Los valores se agrupan en categorías basadas en una lista de umbrales.

    Args:
        data (pd.DataFrame): DataFrame de entrada.
        col (str): Nombre de la columna a procesar.
        thresholds (list[float]): Lista de umbrales para definir los grupos.

    Returns:
        pd.Series: Serie con los valores agrupados.
    """
    porcentaje = data[col].value_counts(normalize=True)

    def asignar_grupo(value):
        freq = porcentaje[value]
        for i, t in enumerate(thresholds):
            if freq < t:
                return f'GRUPO_{i+1}'
        return 'GRUPO_ALTO'

    return data[col].apply(asignar_grupo)


  for col in ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']:
      data[col] = data[col].apply(normalize_text)
  for col in ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']:
      data[col] = group_values_by_frequency(data, col)

  mapeo = {'Si': 1, 'S': 1, 'No': 0, 'N':0}
  target_columns = ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']
  for column in target_columns:
      data[column] = data[column].map(mapeo)
  for col in target_columns:
    data[col] = data[col].fillna(0)

  mapeo_estrato = {'Sin Estrato': 0, 'Estrato 1': 1, 'Estrato 2': 2, 'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6}

  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].map(mapeo_estrato)

  mode_value = data['FAMI_ESTRATOVIVIENDA'].mode()[0]


  data['FAMI_ESTRATOVIVIENDA'] = data['FAMI_ESTRATOVIVIENDA'].fillna(mode_value)

  variables_continuas = ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']

  mapeo_valores_matricula = {
      'Menos de 500 mil': 0,
      'Entre 500 mil y menos de 1 millón': 1,
      'Entre 1 millón y menos de 2.5 millones': 2,
      'Entre 2.5 millones y menos de 4 millones': 3,
      'Entre 4 millones y menos de 5.5 millones': 4,
      'Entre 5.5 millones y menos de 7 millones': 5,
      'Más de 7 millones': 6,
      'No pagó matrícula': 7
  }

  mapeo_valores_horas_trabajo = {
      '0': 0,
      'Menos de 10 horas': 1,
      'Entre 11 y 20 horas': 2,
      'Entre 21 y 30 horas': 3,
      'Más de 30 horas': 4
  }


  data['ESTU_VALORMATRICULAUNIVERSIDAD'] = data['ESTU_VALORMATRICULAUNIVERSIDAD'].map(mapeo_valores_matricula)
  data['ESTU_HORASSEMANATRABAJA'] = data['ESTU_HORASSEMANATRABAJA'].map(mapeo_valores_horas_trabajo)
  for col in variables_continuas:
    data[col] = data[col].fillna(0)

  moda_padre = data['FAMI_EDUCACIONPADRE'].mode()[0]
  moda_madre = data['FAMI_EDUCACIONMADRE'].mode()[0]


  data['FAMI_EDUCACIONPADRE'] = data['FAMI_EDUCACIONPADRE'].fillna(moda_padre)
  data['FAMI_EDUCACIONMADRE'] = data['FAMI_EDUCACIONMADRE'].fillna(moda_madre)
  data = pd.get_dummies(data, columns=[ 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE','ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'], prefix=['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE','ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO'])
  data.replace({True: 1, False: 0})

  return data


In [ ]:
dtr = train
dts = test
lentr = len(dtr)
dtr.shape, dts.shape

((692500, 12), (296786, 12))

In [ ]:
source_cols = [i for i in dtr.columns if i!="RENDIMIENTO_GLOBAL"]
all_data = pd.concat((dtr[source_cols], dts[source_cols]))
all_data.index = range(len(all_data))
all_data = limpiar_y_preprocesar(all_data)

Xtrk, ytrk = all_data.iloc[:lentr].values, dtr["RENDIMIENTO_GLOBAL"].values
Xtsk  = all_data.iloc[lentr:].values

print (Xtrk.shape, ytrk.shape)
print (Xtsk.shape)

In [ ]:
n = int(len(Xtrk)*0.7)
n

484749

In [ ]:
idxs = np.random.permutation(len(Xtrk))
idxs_trm = idxs[:n]
idxs_tsm = idxs[n:]

Xtrm = Xtrk[idxs_trm]
ytrm = ytrk[idxs_trm]

Xtsm = Xtrk[idxs_tsm]
ytsm = ytrk[idxs_tsm]

print (Xtrm.shape, ytrm.shape, Xtsm.shape, ytsm.shape)

(484749, 35) (484749,) (207751, 35) (207751,)


El primer modelo usado es la RegresionLogistica, aunque no es la mejor a través de los resultados de pycarent, sí es rapido de correr y podemos tener un modelo base.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(max_iter=100, n_jobs=-1)
scores = cross_val_score(lr, Xtrk, ytrk, cv=5)
print(scores)

[0.36581227 0.35916245 0.36239711 0.3620722  0.36238989]
